In [4]:
# VERSION 1 - DESCRIPTIVE ANALYSIS
from google.colab import files
uploaded = files.upload()
import pandas as pd
df1 = pd.read_csv('train.csv', low_memory= False)
df2 = pd.read_csv('test.csv', low_memory= False)
# I. Data Cleaning
## Columns Types
df1.info()
df2.info()
## %Blank/null
blank_pct_df1 = df1.isna().sum()
blank_pct_df1 = df2.isna().sum()
df1 = df1.dropna(subset= ['Arrival Delay in Minutes'])
df2 = df2.dropna(subset= ['Arrival Delay in Minutes'])
## %Zero Value
def zero_value_report(df):
  report ={}
  for col in df.select_dtypes(include= 'number').columns:
    zero_pct_df = (df[col] == 0).mean()*100
    if zero_pct_df > 0:
      report[col] = round(zero_pct_df, 2)
  return pd.Series(report, name= 'zero_pct').sort_values(ascending= False)
print(zero_value_report(df1))
print(zero_value_report(df2))
import numpy as np
delay_columns = ['Departure Delay in Minutes', 'Arrival Delay in Minutes']
for col in delay_columns:
  median_val = df1[col].replace(0, np.nan).median()
  df1[col] = df1[col].replace(0, median_val)
  df2[col] = df2[col].replace(0, median_val)
  print((df1[col] == 0).mean() * 100)
  print((df2[col] == 0).mean() * 100)
## Outliers
def outlier_pct(series):
  Q1 = series.quantile(0.25)
  Q3 = series.quantile(0.75)
  IQR = Q3 - Q1
  lower_bound = Q1 - 1.5 * IQR
  upper_bound = Q3 + 1.5 * IQR
  return ((series < lower_bound) | (series > upper_bound)).mean()*100
cols = ['Departure Delay in Minutes', 'Arrival Delay in Minutes']
for col in cols:
  outlier_pct(df1[col])
for col in cols:
  outlier_pct(df2[col])
for col in cols:
  print('train', df1[col].describe())
for col in cols:
  print('/n test', df2[col].describe())
### Grouping
df1['Departure Delay in Minutes_Group'] = pd.cut(df1['Departure Delay in Minutes'], bins= [1, 15, 60, df1['Departure Delay in Minutes'].max()], labels= ['Lower than 15 mins', '15 to 60 mins', 'More than 60 mins'], right= False, include_lowest= True)
df1['Arrival Delay in Minutes_Group'] = pd.cut(df1['Arrival Delay in Minutes'], bins= [1, 15, 60, df1['Arrival Delay in Minutes'].max()], labels= ['Lower than 15 mins', '15 to 60 mins', 'More than 60 mins'], right= False, include_lowest= True)
df2['Departure Delay in Minutes_Group'] = pd.cut(df2['Departure Delay in Minutes'], bins= [1, 15, 60, df2['Departure Delay in Minutes'].max()], labels= ['Lower than 15 mins', '15 to 60 mins', 'More than 60 mins'], right= False, include_lowest= True)
df2['Arrival Delay in Minutes_Group'] = pd.cut(df2['Arrival Delay in Minutes'], bins= [1, 15, 60, df2['Arrival Delay in Minutes'].max()], labels= ['Lower than 15 mins', '15 to 60 mins', 'More than 60 mins'], right= False, include_lowest= True)
# II. Export cleaned data for version 1
df1.to_csv('train_cleaned.csv', index= False)
df2.to_csv('test_cleaned.csv', index= False)
files.download('train_cleaned.csv')
files.download('test_cleaned.csv')

Saving test.csv to test (3).csv
Saving train.csv to train (3).csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103904 entries, 0 to 103903
Data columns (total 25 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   Unnamed: 0                         103904 non-null  int64  
 1   id                                 103904 non-null  int64  
 2   Gender                             103904 non-null  object 
 3   Customer Type                      103904 non-null  object 
 4   Age                                103904 non-null  int64  
 5   Type of Travel                     103904 non-null  object 
 6   Class                              103904 non-null  object 
 7   Flight Distance                    103904 non-null  int64  
 8   Inflight wifi service              103904 non-null  int64  
 9   Departure/Arrival time convenient  103904 non-null  int64  
 10  Ease of Online booking             103

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [6]:
# VERSION 2 - PREDICTIVE ANALYSIS
from google.colab import files
uploaded = files.upload()
import pandas as pd

df1 = pd.read_csv('train_cleaned.csv', low_memory= False)
df2 = pd.read_csv('test_cleaned.csv', low_memory= False)

# Capping outliers
cols = ['Departure Delay in Minutes', 'Arrival Delay in Minutes']
def cap_outliers(series, lower, upper):
  return series.clip(lower, upper)
for col in cols:
  Q1 = df1[col].quantile(0.01)
  Q3 = df1[col].quantile(0.99)
  IQR = Q3 - Q1
  lower_bound = Q1 - 1.5 * IQR
  upper_bound = Q3 + 1.5 * IQR
  df1[f'{col}_capped'] = cap_outliers(df1[col], lower_bound, upper_bound)
  df2[f'{col}_capped'] = cap_outliers(df2[col], lower_bound, upper_bound)

# Export cleaned data for predictive models
df1.to_csv('train_cleaned_v2.csv', index= False)
df2.to_csv('test_cleaned_v2.csv', index= False)
files.download('train_cleaned_v2.csv')
files.download('test_cleaned_v2.csv')

Saving test_cleaned.csv to test_cleaned (2).csv
Saving train_cleaned.csv to train_cleaned (2).csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>